In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install effdet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00

In [ ]:
!pip install timm

In [ ]:
import torch
import torchvision
import os
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet
from torchvision.datasets import CocoDetection
from torch.utils.data import DataLoader
from pycocotools.coco import COCO
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pandas as pd
import json
import os

setting json file

In [ ]:
CLASS_MAP = {"ulcer": 0, "polyp": 1, "cancer": 2}

def csv_to_coco(csv_path, meta_csv, output_json):
    df = pd.read_csv(csv_path)
    meta_df = pd.read_csv(meta_csv)

    df["image_id"] = df["image_id"].astype(str)
    meta_df["image_id"] = meta_df["image_id"].astype(str)

    image_id_map = {row["image_id"]: CLASS_MAP[row["lesion"]] for _, row in df.iterrows()}

    filtered_df = df[df["image_id"].isin(meta_df["image_id"])]

    coco_json = {"images": [], "annotations": [], "categories": []}

    for class_name, class_id in CLASS_MAP.items():
        coco_json["categories"].append({"id": class_id, "name": class_name})

    annotation_id = 1
    images = []
    annotations = []

    for _, row in filtered_df.iterrows():
        image_id = image_id_map[row["image_id"]]
        file_name = row["image_id"]

        if not any(img["id"] == image_id for img in images):
            images.append({
                "id": image_id,
                "file_name": file_name,
                "width": int(row["width"]),
                "height": int(row["height"])
            })

        x_min, y_min, x_max, y_max = row["x_min"], row["y_min"], row["x_max"], row["y_max"]
        bbox = [x_min, y_min, x_max - x_min, y_max - y_min]

        annotations.append({
            "id": annotation_id,
            "image_id": image_id,
            "category_id": CLASS_MAP[row["lesion"]],
            "bbox": bbox,
            "area": bbox[2] * bbox[3],
            "iscrowd": 0
        })
        annotation_id += 1

    coco_json["images"] = images
    coco_json["annotations"] = annotations

    with open(output_json, "w") as f:
        json.dump(coco_json, f, indent=4)

    print(f"COCO JSON saved: {output_json}")


base_path = "/content/drive/MyDrive/Endoscope Detection/colonoscopy"
csv_path = "/content/drive/MyDrive/Endoscope Detection/colonoscopy/resize_colonoscopy.csv"

meta_paths = {
    "train": "/content/drive/MyDrive/Endoscope Detection/colonoscopy/train_meta.csv",
    "val": "/content/drive/MyDrive/Endoscope Detection/colonoscopy/val_meta.csv",
    "test": "/content/drive/MyDrive/Endoscope Detection/colonoscopy/test_meta.csv"
}

json_paths = {
    "train": "/content/drive/MyDrive/Endoscope Detection/colonoscopy/EfficientDet/train_coco.json",
    "val": "/content/drive/MyDrive/Endoscope Detection/colonoscopy/EfficientDet/val_coco.json",
    "test": "/content/drive/MyDrive/Endoscope Detection/colonoscopy/EfficientDet/test_coco.json"
}

for split in ["train", "val", "test"]:
    csv_to_coco(csv_path, meta_paths[split], json_paths[split])



COCO JSON saved: /content/drive/MyDrive/Endoscope Detection/colonoscopy/EfficientDet/train_coco.json
COCO JSON saved: /content/drive/MyDrive/Endoscope Detection/colonoscopy/EfficientDet/val_coco.json
COCO JSON saved: /content/drive/MyDrive/Endoscope Detection/colonoscopy/EfficientDet/test_coco.json


In [ ]:
train_json = "/content/drive/MyDrive/Endoscope Detection/colonoscopy/EfficientDet/train_coco.json"
val_json = "/content/drive/MyDrive/Endoscope Detection/colonoscopy/EfficientDet/val_coco.json"
test_json = "/content/drive/MyDrive/Endoscope Detection/colonoscopy/EfficientDet/test_coco.json"

train_img_dir = "/content/drive/MyDrive/Endoscope Detection/colonoscopy/Dataset/images/train"
val_img_dir = "/content/drive/MyDrive/Endoscope Detection/colonoscopy/Dataset/images/val"
test_img_dir = "/content/drive/MyDrive/Endoscope Detection/colonoscopy/Dataset/images/test"

In [ ]:
train_transforms = A.Compose([
    A.Resize(256, 256),
    ToTensorV2()
])

val_transforms = A.Compose([A.Resize(256,256),
                            ToTensorV2()])


In [ ]:
import numpy as np
class Transforms:
    def __init__(self, transforms: A.Compose):
        self.transforms = transforms

    def __call__(self, img, *args, **kwargs):
        return self.transforms(image=np.array(img))

train_transforms = Transforms(A.Compose([A.Resize(256, 256), ToTensorV2()]))
val_transforms = Transforms(A.Compose([A.Resize(256, 256), ToTensorV2()]))

In [ ]:
class COCODataset(CocoDetection):
    def __init__(self, img_folder, ann_file, transforms=None):
        super().__init__(img_folder, ann_file)
        self.transforms = transforms

    def __getitem__(self, idx):
        img, target = super().__getitem__(idx)
        image_id = self.ids[idx]

        boxes = [obj["bbox"] for obj in target]
        labels = [obj["category_id"] for obj in target]

        if self.transforms:
            transformed = self.transforms(image=img, bboxes=boxes, labels=labels)
            img = transformed["image"]
            boxes = torch.tensor(transformed["bboxes"])
            labels = torch.tensor(transformed["labels"])

        target_dict = {"boxes": boxes, "labels": labels, "image_id": torch.tensor(image_id)}
        return img, target_dict

In [ ]:
train_dataset = COCODataset(train_img_dir, train_json, train_transforms)
val_dataset = COCODataset(val_img_dir, val_json, val_transforms)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [ ]:
config = get_efficientdet_config("tf_efficientdet_d3")
config.num_classes = 3
config.image_size = (256,256)

model = EfficientDet(config, pretrained_backbone=True)
model.class_net = HeadNet(config, num_outputs=config.num_classes)

model = DetBenchTrain(model, config)
model = model.cuda()

In [ ]:
import torch.optim as optim
optimizer = optim.AdamW(model.parameters(), lr = 1e-3)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.8)

In [ ]:
def train_one_epoch(model, train_loader, optimizer):
    model.train()
    total_loss = 0
    for images, targets in train_loader:
        images = torch.stack(images).cuda()
        targets = [{k: v.cuda() for k, v in t.items()} for t in targets]

        optimizer.zero_grad()
        loss_dict = model(images, targets)
        loss = loss_dict["loss"]
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)

In [ ]:
num_epoch = 20

for epoch in range(num_epoch):
    train_loss = train_one_epoch(model, train_loader, optimizer)
    lr_scheduler.step()

    print(f"Epoch {epoch+1}/{num_epoch}, Loss: {train_loss:.4f}")

    if epoch + 1 == num_epoch:
        torch.save(model.state_dict(), "efficientdet_v1.pth")

ValueError: not enough values to unpack (expected 2, got 1)